In [1]:
pwd

'C:\\Users\\Admin\\Jieun\\1019_deeplearning_from_scratch\\ch04_1018'

In [2]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions2 import * 

In [3]:
class SimpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3) # 정규 분포(Noraml Distribution)으로 초기화
    def predict(self, x):
        return np.dot(x, self.W)
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        return loss

In [4]:
net = SimpleNet()
net.W

array([[-0.2167532 ,  0.71529249, -0.63341435],
       [-1.29153399,  1.54186071,  0.68407286]])

In [5]:
x = np.array([0.6, 0.9])
p = net.predict(x)
p

array([-1.29243252,  1.81685013,  0.23561696])

In [6]:
t = np.array([0, 0, 1])
net.loss(x, t)

1.804659446172905

In [7]:
def f(W): return net.loss(x, t)

In [8]:
dW = numerical_gradient(f, net.W)

In [9]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 기울기 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b2'] = np.zeros(output_size)
    def predict(self, x):
        W1, W2 = self.params['W1']. self.params['W2']
        b1, b2 = self.params['b1']. self.params['b2']
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W1) + b2
        z2 = sigmoid(a2)
        y = softmax(a2)
        return y
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        return loss
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        accuracy = np.sum(y==t)/float(x.shape[0])
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        # 아래에 코드 채우세요
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        W1, W2 = self.params['b1'], self.params['b2']
        grads = {}
        batch_num = x.shape[0]
        # forward
#             기존 코드 참고하시면 2개의 레이어가 있어요
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        # backward
        dy = (y-t) / batch_num # 마지막 레이어의 델타
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)
        return grads

In [10]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
print(net.params['W1'].shape) # (784,100)
print(net.params['b1'].shape) # (100,)
print(net.params['W2'].shape) # (100,10)
print(net.params['b2'].shape) # (10,)

(784, 100)
(100,)
(784, 100)
(10,)


<a href='https://m.blog.naver.com/PostView.nhn?isHttpsRedirect=true&blogId=laonple&logNo=220507299181'>역전파의 개념</a>

<img src='images/back.jpg' />

## 무시무시 어려운 back propagation

$$ C = \frac{1}{2n}\sum_{x} (y(x) - a^{L}(x))^{2}  $$

<img src='images/cute.jpg' />

$$ \delta_{j}^{l} \equiv \frac{\partial C}{\partial{z}_{j}^{l}} $$

$$ \delta_{j}^{L} \equiv \frac{\partial C}{\partial{a}_{j}^{L}} \sigma^{'}(z_{j}^{L}) $$ 

$$ C = \frac{1}{2n}\sum_{x} (y(x) - a^{L}(x))^{2}  $$
↑ 요거 미분하면 ↓
$$ \frac{\partial C}{\partial{a}_{j}^{L}} = (a_{j}^{L} - y_{j}) $$ 

$$ \frac{\partial C}{\partial{b}_{j}^{l}} = \delta_{j}^{l} $$
$$ \frac{\partial C}{\partial{w_{jk}^{l}}} = a_{k}^{l-2} \delta_{j}^{l} $$
$$ \frac{\partial C}{\partial \omega} = a_{in} \delta_{out} $$

<mark>☆ 하이라이트 ☆</mark>
<img src='images/very_important.jpg' />

<a href='http://neuralnetworksanddeeplearning.com/chap2.html'>역전파</a>

<img src='images/summary.jpg'/>

## 미니 배치 학습 구현하기

In [11]:
from common.two_layer_net import TwoLayerNet
from dataset.mnist import load_mnist

In [12]:
# 데이터 읽기
(x_train, t_train), (x_test, y_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

In [13]:
# 하이퍼 파라미터
iters_num = 10000 # 반복 횟수를 적절히 설정
train_size = x_train.shape[0]
batch_size = 100 # 미니 배치 크기
learing_rate = 0.1

In [14]:
tarin_loss_list = []
train_acc_list = []
test_acc_list = []
# 1 epoch 당 반복수
iter_per_epoch = max(train_size / batch_size, 1)
for i in range(iters_num):
    # 미니 배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    # 60000개의 데이터 중 랜덤으로 batch_size만큼 데이터를 학습함.
    # 이것을 확률적 경사 하강법 (Stochastic Gradient Descent SGD)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    # 매개변수 갱신
    for key in ('W1', 'b1','W2', 'b2'):
        network.params[key] -= learing_rate * grad[key]
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    # 1 epoch 당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + " , " + str(test_acc))
# 그래프 그리기
markers = {'train:':'o', 'test':'s'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

NameError: name 'numerical_gradient' is not defined